In [1]:
import numpy as np
import scipy as sp
import math, sys
from sklearn import datasets, base
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
import pandas as pd
from sklearn.cross_validation import cross_val_score
from copy import deepcopy
import itertools
sys.setrecursionlimit(1500)

classifiers = [
    KNeighborsClassifier(n_jobs = -1),
    SVC(probability = True),
    DecisionTreeClassifier(),
    RandomForestClassifier()]

classifiers_name = ["KNN", "SVM", "DT", "RF"]
beta = "RF"

parameters = {"KNN":{"n_neighbors":[]},
              "SVM":{"C":[],"kernel":[],"degree":[]},
              "DT":{"splitter":[], "max_depth":[]},
              "RF":{"n_estimators":[], "max_depth":[] },
            }
classifier_parameters = []

iris = datasets.load_iris()
X = iris.data
Y = iris.target
Df = pd.DataFrame()
Df["X"] = X.tolist()
Df["Y"] = Y.tolist()

/home/neo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def sample_hp_Ai(Ai):
    global classifier_parameters
    if(Ai is "KNN"):
        parameters["KNN"]["n_neighbors"] = math.ceil(np.random.uniform(20))
        classifier_parameters.append(parameters["KNN"].copy())
        return(parameters["KNN"])
    elif(Ai is "SVM"):
        parameters["SVM"]["C"] = float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0))))
        parameters["SVM"]["kernel"] = str(np.random.choice(["linear", "rbf"]))
        parameters["SVM"]["degree"] = np.random.randint(1,3)
        classifier_parameters.append(parameters["SVM"].copy())
        return(parameters["SVM"])
    elif(Ai is "DT"):
        parameters["DT"]["splitter"] = np.random.choice(["best", "random"])
        parameters["DT"]["max_depth"] = np.random.randint(1,10)
        classifier_parameters.append(parameters["DT"].copy())
        return(parameters["DT"])
    elif(Ai is "RF"):
        parameters["RF"]["n_estimators"] = np.random.randint(1,20)
        parameters["RF"]["max_depth"] = np.random.randint(1,10)
        classifier_parameters.append(parameters["RF"].copy())
        return(parameters["RF"])
    elif(Ai is "XGB"):
        parameters["XGB"]["learning_rate"].append(float("{0:.2f}".format(abs(np.random.normal(loc=1.0, scale=1.0)))))
        parameters["XGB"]["n_estimators"].append(np.random.randint(100,200))
        parameters["XGB"]["max_depth"].append(np.random.randint(5,10))
        parameters["XGB"]["min_child_weight"].append(np.random.randint(1,10))
        classifier_parameters.append(parameters["XGB"].copy())
        return(parameters["XGB"])

In [3]:
def get_params(A, B, P, N):
    mu = np.random.dirichlet(P)
    N = [int(mu_i*N) for mu_i in mu]
    [sample_hp_Ai(classifiers_name[i]) for i in range(len(classifiers_name)) for j in range(N[i])]
    Phi = sample_hp_Ai(beta)
    Psi = [parameters[key] for key in parameters if key is not "XGB"]
    return Phi, N, Psi

In [4]:
class Blend:
    def __init__(self, A = ["KNN", "SVM", "GP", "DT", "RF"], B = "XGB",D = [], L = 2, Phi = [], Psi = [], N=100 ):
        self.A = A
        self.B = B
        self.D = D
        self.L = L
        self.Phi = Phi
        self.Psi = Psi
        self.N = N
        self.M = None
        self.F = None
        self.G = None
        
    def fit(self, A, B, D, L, Phi, N, Psi):
#         p = np.random.normal(loc=0.7, scale=0.1)
        p=0.7
        D_fw = pd.DataFrame()
        M = []
        F = []
        G = []
        for l in range(1,L+1):
            D_dash = D.sample(frac=p)
            orig_index = sorted(Df.index.tolist())
            train_index = sorted(D_dash.index.tolist())
            test_index = sorted(set(orig_index) - set(train_index))
            D_dash_bar = Df.ix[test_index]
            classifier_index = 0
            for i in range(0,len(classifiers_name)):
                temp = []
                if(N[i]):
                    for j in range(0,N[i]):
                        temp_clf = None
                        temp_clf = classifiers[i].set_params(**classifier_parameters[classifier_index])
                        temp_clf = temp_clf.fit(X = D_dash["X"].tolist(), y = D_dash["Y"].tolist())
                        temp.append(deepcopy(temp_clf))
                        classifier_index += 1
                M.append(temp)
                self.M = M
            for row in D_dash_bar["X"]:
                F_temp = []
                G_temp = []
                for models in M:
                    for model in models:
                        M_bc = model.predict_proba([row])
                        F_temp += [xa*ele for xa in row for ele in M_bc[0]]
                        G_temp += [ele for ele in M_bc[0]]
                F.append(F_temp)
                G.append(G_temp)
                self.F = F
                self.G = G
            d_assign = [list(itertools.chain.from_iterable(ele)) for ele in \
                        list(zip(D_dash_bar["X"].tolist().copy(),F.copy(),G.copy()))]
            D_fw["X"] = d_assign
            D_fw["Y"] = D_dash_bar["Y"].tolist().copy()
            D_fw = D_fw.reset_index()
            
            XX = np.asarray([np.asarray(row) for row in D_fw["X"]])
            YY = np.asarray([np.asarray(row) for row in D_fw["Y"]])
            
            Beta = RandomForestClassifier()
            Beta.set_params(**classifier_parameters[-1])
            Beta = Beta.fit(X=XX, y=YY)   
            self.B = Beta
        return(Beta)
    
    def predict(self, X):
        return (self.B.predict(X))
    
    def score(self, A, B, D, L, Phi, N, Psi):
        p=1
        D_fw = pd.DataFrame()
        M = []
        F = []
        G = []
        for l in range(1,L+1):
            D_dash = D.sample(frac=p)
            
            for row in D_dash["X"]:
                F_temp = []
                G_temp = []
                for models in self.M:
                    for model in models:
                        M_bc = model.predict_proba([row])
                        F_temp += [xa*ele for xa in row for ele in M_bc[0]]
                        G_temp += [ele for ele in M_bc[0]]
                F.append(F_temp)
                G.append(G_temp)
            d_assign = [list(itertools.chain.from_iterable(ele)) for ele in \
                        list(zip(D_dash["X"].tolist().copy(),F.copy(),G.copy()))]
            D_fw["X"] = d_assign
            D_fw["Y"] = D_dash["Y"].tolist().copy()
            D_fw = D_fw.reset_index()
            
            XX = np.asarray([np.asarray(row) for row in D_fw["X"]])
            YY = np.asarray([np.asarray(row) for row in D_fw["Y"]])
            score = self.B.score(X=XX, y=YY)
            return score

In [6]:
def blending_ensemble(A, B, D, P, N, k, R):
    L = 2
    M = []    
    er = []
    Phi_list = []
    Nr_list = []
    Psi_list = []
    for r in range(1,R+1):
        print("r=",r)
        Phi_r, N_r, Psi_r = get_params(A, B, P, N=10)
        k_fold = KFold(k)
        scores = []
        for k, (train, test) in enumerate(k_fold.split(Df["X"], Df["Y"])):
            print("k=",k)
            b = None
            b = Blend()
            D_train = pd.DataFrame()
            D_test = pd.DataFrame()
            D_train["X"] = Df["X"][train]
            D_train["Y"] = Df["Y"][train]
            D_test["X"] = Df["X"][test]
            D_test["Y"] = Df["Y"][test]
            
            b.fit(A=classifiers_name, B="RF", D=D_train, L=1, Phi=Phi_r, N=N_r, Psi=Psi_r)
            score = b.score(A=classifiers_name, B="RF", D=D_test, \
                                                      L=1, Phi=Phi_r, N=N_r, Psi=Psi_r)
            scores.append(score.copy())
            
        mean = np.mean(scores)
        print("Mean score at r={0} is: {1}".format(r,mean))
        
        er.append(mean)
        Phi_list.append(Phi_r.copy())
        Nr_list.append(N_r.copy())
        Psi_list.append(Psi_r.copy())
    best_r = max(er)
    best_index = er.index(best_r)
    print("Best performance is at the point r={0} with",
    "Blend function model parameters for {1}",
    "Base algorithm mixing numbers: {2}",
    "Individual algorithm model parameters:{3}"\
         .format(r,Phi_list[best_index], Nr_list[best_index], Psi_list[best_index]))

In [7]:
b = blending_ensemble(A=classifiers_name, B="RF", D=Df, P=(np.arange(1,len(classifiers_name)+1)*100)[::-1], \
                  N=3, k=3, R=1)

/home/neo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


KeyboardInterrupt: 

In [ ]:
b

In [ ]:
# Phi, N, Psi = get_params(classifiers_name, "RF", P=(np.arange(1,len(classifiers_name)+1)*100)[::-1], N=10)
# b =Blend()
# Beta = b.fit(A=classifiers_name, B="RF", D=Df, L=1, Phi=Phi, N=N, Psi=Psi)

In [ ]:
a = [1,2,3,1]

In [ ]:
a.index(max(a))